<a href="https://colab.research.google.com/github/jvishnuvardhan/TF_Lite/blob/master/TFlite_Yolov3_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf-nightly

     |████████████████████████████████| 520.9MB 29kB/s 
     |████████████████████████████████| 2.9MB 42.7MB/s 
     |████████████████████████████████| 460kB 46.8MB/s 


In [0]:
import tensorflow as tf
tf.__version__

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!unzip /content/drive/My\ Drive/TFLite_models/savedmodel.zip -d /content

In [0]:
import tensorflow as tf
# Note: full quantization requires tf 1.15 or higher
import numpy as np
from PIL import Image

def representative_dataset_gen():
  data = np.random.rand(100,416,416,3).astype(dtype=np.float32)
  for i in range(num_calibration_steps):
    # Get sample input data as a numpy array in a method of your choosing.
    # image, = data.take(1)
    image = np.expand_dims(data[i,:,:,:], axis=0)
    yield [image]

num_calibration_steps = 100
saved_model_dir = '/content/savedmodel/yolov3/1' # dir of YOLOv3 tf implementation

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.experimental_new_converter =True
converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# converter.representative_dataset = tf.lite.RepresentativeDataset(representative_dataset_gen)
converter.representative_dataset = representative_dataset_gen

# Enforce full integer quantization for all ops and use int input/output
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
#converter.allow_custom_ops = True
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_quant_model = converter.convert()
open("./converted_model.tflite", "wb").write(tflite_quant_model)